## Step 0: Latent Dirichlet Allocation ##

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

## Step 1: Load the dataset

The dataset we'll use is a list of over one million news headlines published over a period of 15 years. We'll start by loading it from the `abcnews-date-text.csv` file.

In [3]:
'''
Load the dataset from the CSV and save it to 'data_text'
'''
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
# We only need the Headlines text column from the data
data_text = data[:300000][['headline_text']];
data_text['index'] = data_text.index

documents = data_text

Let's glance at the dataset:

In [4]:
'''
Get the total number of documents
'''
print(len(documents))

300000


In [5]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.


In [6]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [11]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lveys\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

### Lemmatizer Example
Before preprocessing our dataset, let's first look at an lemmatizing example. What would be the output if we lemmatized the word 'went':

In [12]:
print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense

go


### Stemmer Example
Let's also look at a stemming example. Let's throw a number of words at the stemmer and see how it deals with each one:

In [15]:
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [16]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            # TODO: Apply lemmatize_stemming() on the token, then add to the results list
            result.append(lemmatize_stemming(token))
            
    return result



In [19]:
'''
Preview a document after preprocessing
'''
document_num = 4310
doc_sample = documents[documents['index'] == document_num].values[0][0]

doc_sample

'rain helps dampen bushfires'

In [17]:
print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [21]:
doc_sample = 'the doctor came to the village to visit some patients yesterday.'
print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['the', 'doctor', 'came', 'to', 'the', 'village', 'to', 'visit', 'some', 'patients', 'yesterday.']


Tokenized and lemmatized document: 
['doctor', 'come', 'villag', 'visit', 'patient', 'yesterday']


In [20]:
documents

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4
...,...,...
299995,broughton hall audit reveals serious breaches,299995
299996,broughton hall fails key standards,299996
299997,broughton hall safe for residents govt says,299997
299998,burn off at conservation park aims to prevent,299998


Let's now preprocess all the news headlines we have. To do that, let's use the [map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function from pandas to apply `preprocess()` to the `headline_text` column

**Note**: This may take a few minutes (it take 6 minutes on my laptop)

In [24]:
# TODO: preprocess all the headlines, saving the list of results as 'processed_docs'
processed_docs = documents['headline_text'].map(lambda x: preprocess(x))

In [25]:
'''
Preview 'processed_docs'
'''
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

## Step 3.1: Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass `processed_docs` to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) and call it '`dictionary`'.

In [26]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

In [27]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [29]:
print(dictionary)

Dictionary(24903 unique tokens: ['broadcast', 'communiti', 'decid', 'licenc', 'awar']...)


In [30]:
from collections import Counter
count = Counter()
for doc in processed_docs:
    for word in doc:
        count[word]+=1
print(count)

Counter({'polic': 11610, 'govt': 10074, 'plan': 8867, 'council': 6982, 'urg': 6039, 'say': 5388, 'kill': 5198, 'charg': 5058, 'fund': 4735, 'court': 4563, 'face': 4448, 'warn': 4415, 'water': 4336, 'iraq': 4211, 'seek': 4201, 'claim': 4198, 'report': 3907, 'death': 3743, 'crash': 3693, 'boost': 3493, 'group': 3437, 'attack': 3394, 'chang': 3175, 'health': 3154, 'hospit': 3072, 'help': 3051, 'continu': 3022, 'open': 2926, 'miss': 2921, 'servic': 2844, 'call': 2769, 'consid': 2763, 'rise': 2717, 'minist': 2714, 'investig': 2700, 'jail': 2667, 'concern': 2667, 'probe': 2662, 'australia': 2625, 'test': 2622, 'drug': 2580, 'murder': 2567, 'fear': 2505, 'lead': 2424, 'world': 2417, 'hous': 2408, 'sydney': 2407, 'worker': 2401, 'reject': 2396, 'nation': 2390, 'work': 2360, 'road': 2358, 'secur': 2355, 'home': 2350, 'meet': 2344, 'accus': 2340, 'talk': 2330, 'year': 2305, 'support': 2287, 'school': 2287, 'win': 2248, 'protest': 2241, 'power': 2233, 'offer': 2230, 'push': 2218, 'australian': 22

In [36]:
print(count['communiti'])

1668


In [31]:
len(count)

24903

** Gensim filter_extremes **

[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [32]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
# TODO: apply dictionary.filter_extremes() with the parameters mentioned above
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=100000)

In [37]:
len(dictionary)

6718

** Gensim doc2bow **

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [33]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
# TODO
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [34]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
bow_corpus[document_num]

[(71, 1), (107, 1), (462, 1), (3530, 1)]

In [35]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 71 ("bushfir") appears 1 time.
Word 107 ("help") appears 1 time.
Word 462 ("rain") appears 1 time.
Word 3530 ("dampen") appears 1 time.


## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [38]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models

# TODO
tfidf = models.TfidfModel(bow_corpus)

In [39]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
# TODO
corpus_tfidf = tfidf[bow_corpus]

In [40]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5959813347777092),
 (1, 0.39204529549491984),
 (2, 0.48531419274988147),
 (3, 0.5055461098578569)]


## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [41]:
# LDA mono-core -- fallback code in case LdaMulticore throws an error on your machine
# lda_model = gensim.models.LdaModel(bow_corpus, 
#                                    num_topics = 10, 
#                                    id2word = dictionary,                                    
#                                    passes = 50)

# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
# TODO
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics = 10, id2word = dictionary, passes = 50)

In [45]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.022*"world" + 0.021*"win" + 0.018*"take" + 0.018*"test" + 0.016*"strike" + 0.015*"lead" + 0.013*"action" + 0.012*"south" + 0.012*"award" + 0.012*"worker"


Topic: 1 
Words: 0.036*"report" + 0.034*"urg" + 0.029*"help" + 0.017*"inquiri" + 0.013*"blaze" + 0.013*"famili" + 0.013*"govt" + 0.012*"public" + 0.011*"abus" + 0.011*"releas"


Topic: 2 
Words: 0.038*"kill" + 0.029*"crash" + 0.029*"iraq" + 0.017*"continu" + 0.015*"bomb" + 0.014*"coast" + 0.014*"attack" + 0.014*"die" + 0.013*"sydney" + 0.013*"dead"


Topic: 3 
Words: 0.042*"plan" + 0.033*"council" + 0.026*"govt" + 0.022*"fund" + 0.022*"water" + 0.016*"group" + 0.013*"servic" + 0.012*"boost" + 0.012*"concern" + 0.012*"urg"


Topic: 4 
Words: 0.024*"say" + 0.017*"labor" + 0.016*"elect" + 0.016*"protest" + 0.016*"minist" + 0.016*"defend" + 0.014*"govt" + 0.013*"chief" + 0.012*"leader" + 0.011*"howard"


Topic: 5 
Words: 0.046*"warn" + 0.022*"fight" + 0.014*"threat" + 0.014*"nuclear" + 0.013*"prompt" + 0.013*"spark" +

### Classification of the topics ###

Using the words in each topic and their corresponding weights, what categories were you able to infer?

* 0: social, world news
* 1: Urgent issue / catastrophe
* 2: Accidents, wars, conflicts
* 3: Economy
* 4: Politics
* 5: Diplomacy
* 6: Agriculture
* 7: Sports
* 8: Justice
* 9: Claims, disputes

## Step 4.2 Running LDA using TF-IDF ##

In [43]:
'''
Define lda model using corpus_tfidf, again using gensim.models.LdaMulticore()
'''
# TODO
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics = 10, id2word = dictionary, passes = 50)

In [50]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} \nWord: {}".format(idx, topic))
    print("\n")

Topic: 0 
Word: 0.015*"rise" + 0.013*"market" + 0.013*"price" + 0.009*"rate" + 0.008*"sale" + 0.008*"profit" + 0.007*"rat" + 0.007*"high" + 0.007*"record" + 0.006*"share"


Topic: 1 
Word: 0.021*"closer" + 0.009*"law" + 0.008*"govt" + 0.008*"polici" + 0.006*"sack" + 0.006*"liber" + 0.005*"urg" + 0.005*"council" + 0.005*"anti" + 0.005*"say"


Topic: 2 
Word: 0.010*"win" + 0.009*"world" + 0.009*"england" + 0.007*"aussi" + 0.007*"australia" + 0.007*"tour" + 0.007*"test" + 0.006*"award" + 0.005*"team" + 0.005*"match"


Topic: 3 
Word: 0.022*"crash" + 0.015*"miss" + 0.014*"polic" + 0.014*"die" + 0.014*"search" + 0.012*"death" + 0.011*"investig" + 0.010*"road" + 0.009*"accid" + 0.008*"victim"


Topic: 4 
Word: 0.008*"thousand" + 0.007*"steal" + 0.007*"festiv" + 0.007*"alic" + 0.006*"hobart" + 0.006*"celebr" + 0.006*"shop" + 0.006*"hunt" + 0.006*"turn" + 0.006*"polic"


Topic: 5 
Word: 0.027*"kill" + 0.017*"iraq" + 0.016*"bomb" + 0.013*"attack" + 0.011*"blast" + 0.010*"troop" + 0.010*"iraqi" 

### Classification of the topics ###

As we can see, when using tf-idf, heavier weights are given to words that are not as frequent which results in nouns being factored in. That makes it harder to figure out the categories as nouns can be hard to categorize. This goes to show that the models we apply depend on the type of corpus of text we are dealing with. 

Using the words in each topic and their corresponding weights, what categories could you find?

* 0: Business
* 1: Politics
* 2: Sports - England vs Australia game
* 3: Accident report
* 4: Tourism
* 5: Terrorism
* 6: Internal Policy, internal issues
* 7: Justice
* 8: Competition?
* 9: Social issues

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model

We will check to see where our test document would be classified. 

In [46]:
'''
Text of sample document 4310
'''
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [47]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''
document_num = 4310
# Our test document is document number 4310

# TODO
# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.42005881667137146	 
Topic: 0.036*"report" + 0.034*"urg" + 0.029*"help" + 0.017*"inquiri" + 0.013*"blaze" + 0.013*"famili" + 0.013*"govt" + 0.012*"public" + 0.011*"abus" + 0.011*"releas"

Score: 0.41991540789604187	 
Topic: 0.019*"rise" + 0.019*"school" + 0.017*"farmer" + 0.017*"price" + 0.014*"market" + 0.014*"record" + 0.013*"drought" + 0.013*"high" + 0.013*"deal" + 0.012*"rain"

Score: 0.020003225654363632	 
Topic: 0.022*"world" + 0.021*"win" + 0.018*"take" + 0.018*"test" + 0.016*"strike" + 0.015*"lead" + 0.013*"action" + 0.012*"south" + 0.012*"award" + 0.012*"worker"

Score: 0.020003225654363632	 
Topic: 0.038*"kill" + 0.029*"crash" + 0.029*"iraq" + 0.017*"continu" + 0.015*"bomb" + 0.014*"coast" + 0.014*"attack" + 0.014*"die" + 0.013*"sydney" + 0.013*"dead"

Score: 0.020003225654363632	 
Topic: 0.042*"plan" + 0.033*"council" + 0.026*"govt" + 0.022*"fund" + 0.022*"water" + 0.016*"group" + 0.013*"servic" + 0.012*"boost" + 0.012*"concern" + 0.012*"urg"

Score: 0.0200032256543

### It has the highest probability (`x`) to be  part of the topic that we assigned as X, which is the accurate classification. ###

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model

In [48]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
'''
# Our test document is document number 4310
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.8199642896652222	 
Topic: 0.012*"govt" + 0.012*"water" + 0.011*"plan" + 0.011*"fund" + 0.011*"council" + 0.008*"health" + 0.008*"boost" + 0.008*"urg" + 0.007*"servic" + 0.007*"group"

Score: 0.020005038008093834	 
Topic: 0.015*"rise" + 0.013*"market" + 0.013*"price" + 0.009*"rate" + 0.008*"sale" + 0.008*"profit" + 0.007*"rat" + 0.007*"high" + 0.007*"record" + 0.006*"share"

Score: 0.02000478468835354	 
Topic: 0.022*"crash" + 0.015*"miss" + 0.014*"polic" + 0.014*"die" + 0.014*"search" + 0.012*"death" + 0.011*"investig" + 0.010*"road" + 0.009*"accid" + 0.008*"victim"

Score: 0.020004048943519592	 
Topic: 0.028*"charg" + 0.024*"court" + 0.019*"polic" + 0.017*"murder" + 0.015*"jail" + 0.014*"face" + 0.014*"drug" + 0.010*"assault" + 0.010*"accus" + 0.009*"appeal"

Score: 0.02000395581126213	 
Topic: 0.008*"thousand" + 0.007*"steal" + 0.007*"festiv" + 0.007*"alic" + 0.006*"hobart" + 0.006*"celebr" + 0.006*"shop" + 0.006*"hunt" + 0.006*"turn" + 0.006*"polic"

Score: 0.02000384777784

### It has the highest probability (`x%`) to be  part of the topic that we assigned as X. ###

## Step 6: Testing model on unseen document ##

In [68]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
threshold = 0.1
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\n Topic: {}".format(score, lda_model.print_topic(index, 5)))
    if score < threshold: break
    print()

Score: 0.619922399520874
 Topic: 0.022*"world" + 0.021*"win" + 0.018*"take" + 0.018*"test" + 0.016*"strike"

Score: 0.220015749335289
 Topic: 0.020*"australia" + 0.015*"final" + 0.014*"open" + 0.012*"play" + 0.011*"england"

Score: 0.020012229681015015
 Topic: 0.042*"plan" + 0.033*"council" + 0.026*"govt" + 0.022*"fund" + 0.022*"water"


The model correctly classifies the unseen document with 'x'% probability to the X category.

In [78]:
unseen_document = "The US presidential election are scheduled in November this year."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
threshold = 0.1
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\n Topic: {}".format(score, lda_model.print_topic(index, 5)))
    if score < threshold: break
    print()

Score: 0.5166441798210144
 Topic: 0.024*"say" + 0.017*"labor" + 0.016*"elect" + 0.016*"protest" + 0.016*"minist"

Score: 0.183380588889122
 Topic: 0.046*"warn" + 0.022*"fight" + 0.014*"threat" + 0.014*"nuclear" + 0.013*"prompt"

Score: 0.1832539439201355
 Topic: 0.022*"world" + 0.021*"win" + 0.018*"take" + 0.018*"test" + 0.016*"strike"

Score: 0.016678575426340103
 Topic: 0.019*"rise" + 0.019*"school" + 0.017*"farmer" + 0.017*"price" + 0.014*"market"


In [72]:
unseen_document = "Thousands of new entrepreneurs starting businesses."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
threshold = 0.1
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\n Topic: {}".format(score, lda_model.print_topic(index, 5)))
    if score < threshold: break
    print()

Score: 0.27500542998313904
 Topic: 0.042*"plan" + 0.033*"council" + 0.026*"govt" + 0.022*"fund" + 0.022*"water"

Score: 0.27499648928642273
 Topic: 0.038*"kill" + 0.029*"crash" + 0.029*"iraq" + 0.017*"continu" + 0.015*"bomb"

Score: 0.27496233582496643
 Topic: 0.020*"australia" + 0.015*"final" + 0.014*"open" + 0.012*"play" + 0.011*"england"

Score: 0.025009064003825188
 Topic: 0.022*"world" + 0.021*"win" + 0.018*"take" + 0.018*"test" + 0.016*"strike"
